# Clustering Methods

Unsupervised learning: K-Means and Hierarchical Clustering.

## Contents
1. K-Means Clustering
2. Effect of Initialization (nstart)
3. Hierarchical Clustering
4. Linkage Methods (Complete, Average, Single)
5. Scaling and Distance Metrics
6. Application to Gene Expression Data (NCI60)

## Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist, squareform
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 1. K-Means Clustering

Partition observations into K clusters.

In [ ]:
# Generate synthetic data with 2 true clusters
np.random.seed(2)

X = np.random.randn(50, 2)
X[:25, 0] = X[:25, 0] + 3  # Shift first cluster
X[:25, 1] = X[:25, 1] - 4

# True labels (for visualization only)
true_labels = np.array([0]*25 + [1]*25)

print(f"Data shape: {X.shape}")
print(f"Two clusters: 25 observations each")

In [ ]:
# Visualize true clusters
plt.figure(figsize=(8, 6))
plt.scatter(X[true_labels == 0, 0], X[true_labels == 0, 1], 
           c='red', s=100, alpha=0.6, label='True Cluster 1')
plt.scatter(X[true_labels == 1, 0], X[true_labels == 1, 1], 
           c='blue', s=100, alpha=0.6, label='True Cluster 2')
plt.xlabel('X1', fontsize=12)
plt.ylabel('X2', fontsize=12)
plt.title('True Cluster Structure', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Fit K-Means with K=2
np.random.seed(2)
kmeans = KMeans(n_clusters=2, n_init=20, random_state=2)
kmeans.fit(X)

print("K-Means Clustering (K=2):")
print(f"Cluster assignments (first 10): {kmeans.labels_[:10]}")
print(f"\nCluster centers:")
print(kmeans.cluster_centers_)
print(f"\nWithin-cluster sum of squares: {kmeans.inertia_:.2f}")

In [ ]:
# Visualize K-Means results
plt.figure(figsize=(8, 6))
plt.scatter(X[kmeans.labels_ == 0, 0], X[kmeans.labels_ == 0, 1],
           c='red', s=100, alpha=0.6, label='Cluster 1')
plt.scatter(X[kmeans.labels_ == 1, 0], X[kmeans.labels_ == 1, 1],
           c='blue', s=100, alpha=0.6, label='Cluster 2')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
           c='black', s=300, marker='X', edgecolors='yellow', linewidths=3,
           label='Centroids')
plt.xlabel('X1', fontsize=12)
plt.ylabel('X2', fontsize=12)
plt.title('K-Means Clustering Results with K=2', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Check if clusters match true labels
accuracy = max(np.mean(kmeans.labels_ == true_labels), 
              np.mean(kmeans.labels_ != true_labels))  # Account for label switching
print(f"Clustering accuracy: {accuracy*100:.1f}%")

### K-Means with K=3

In [ ]:
# Fit K-Means with K=3 (wrong number of clusters)
np.random.seed(4)
kmeans_3 = KMeans(n_clusters=3, n_init=20, random_state=4)
kmeans_3.fit(X)

print("K-Means Clustering (K=3):")
print(f"Cluster sizes: {np.bincount(kmeans_3.labels_)}")
print(f"Within-cluster sum of squares: {kmeans_3.inertia_:.2f}")
print(f"\nCluster centers:")
print(kmeans_3.cluster_centers_)

In [ ]:
# Visualize K=3 results
plt.figure(figsize=(8, 6))
colors = ['red', 'blue', 'green']
for i in range(3):
    mask = kmeans_3.labels_ == i
    plt.scatter(X[mask, 0], X[mask, 1], c=colors[i], s=100, alpha=0.6, 
               label=f'Cluster {i+1}')
plt.scatter(kmeans_3.cluster_centers_[:, 0], kmeans_3.cluster_centers_[:, 1],
           c='black', s=300, marker='X', edgecolors='yellow', linewidths=3,
           label='Centroids')
plt.xlabel('X1', fontsize=12)
plt.ylabel('X2', fontsize=12)
plt.title('K-Means Clustering Results with K=3', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Observation: K=3 splits one of the true clusters")

## 2. Effect of Initialization (n_init)

K-Means can get stuck in local optima. Use multiple initializations.

In [ ]:
# Single initialization (n_init=1)
np.random.seed(3)
kmeans_single = KMeans(n_clusters=3, n_init=1, random_state=3)
kmeans_single.fit(X)

# Multiple initializations (n_init=20)
np.random.seed(3)
kmeans_multiple = KMeans(n_clusters=3, n_init=20, random_state=3)
kmeans_multiple.fit(X)

print("Effect of n_init:")
print(f"\nn_init=1:  Total within-cluster SS = {kmeans_single.inertia_:.2f}")
print(f"n_init=20: Total within-cluster SS = {kmeans_multiple.inertia_:.2f}")

print(f"\nImprovement: {((kmeans_single.inertia_ - kmeans_multiple.inertia_) / kmeans_single.inertia_ * 100):.1f}%")
print(f"\nRecommendation: Always use n_init >= 20 to avoid local optima!")

In [ ]:
# Compare different n_init values
n_init_values = [1, 5, 10, 20, 50, 100]
wcss_values = []

for n_init in n_init_values:
    np.random.seed(3)
    kmeans_temp = KMeans(n_clusters=3, n_init=n_init, random_state=3)
    kmeans_temp.fit(X)
    wcss_values.append(kmeans_temp.inertia_)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(n_init_values, wcss_values, marker='o', linewidth=2, markersize=8)
plt.xlabel('n_init (number of initializations)', fontsize=12)
plt.ylabel('Total Within-Cluster Sum of Squares', fontsize=12)
plt.title('Effect of Multiple Initializations on K-Means', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("More initializations → better chance of finding global optimum")

## 3. Hierarchical Clustering

Build a hierarchy of clusters (bottom-up).

In [ ]:
# Compute linkage matrices for different methods
linkage_complete = linkage(X, method='complete')  # Maximum distance
linkage_average = linkage(X, method='average')    # Average distance
linkage_single = linkage(X, method='single')      # Minimum distance

print("Hierarchical Clustering Linkage Methods:")
print("\n1. Complete Linkage: Max distance between clusters")
print("2. Average Linkage: Average distance between clusters")
print("3. Single Linkage: Min distance between clusters")

### Dendrograms

In [ ]:
# Plot dendrograms
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Complete linkage
dendrogram(linkage_complete, ax=axes[0], no_labels=True)
axes[0].set_title('Complete Linkage', fontsize=14)
axes[0].set_xlabel('')
axes[0].set_ylabel('Distance', fontsize=12)

# Average linkage
dendrogram(linkage_average, ax=axes[1], no_labels=True)
axes[1].set_title('Average Linkage', fontsize=14)
axes[1].set_xlabel('')
axes[1].set_ylabel('Distance', fontsize=12)

# Single linkage
dendrogram(linkage_single, ax=axes[2], no_labels=True)
axes[2].set_title('Single Linkage', fontsize=14)
axes[2].set_xlabel('')
axes[2].set_ylabel('Distance', fontsize=12)

plt.tight_layout()
plt.show()

print("Dendrogram Interpretation:")
print("- Height = distance at which clusters merge")
print("- Cut horizontally to get clusters")
print("- Single linkage tends to produce 'chaining'")

### Cutting Dendrograms to Get Clusters

In [ ]:
# Cut dendrogram at different heights to get K clusters
clusters_complete_2 = fcluster(linkage_complete, 2, criterion='maxclust')
clusters_average_2 = fcluster(linkage_average, 2, criterion='maxclust')
clusters_single_2 = fcluster(linkage_single, 2, criterion='maxclust')
clusters_single_4 = fcluster(linkage_single, 4, criterion='maxclust')

print("Cluster assignments (first 10 observations):")
print(f"\nComplete linkage (K=2): {clusters_complete_2[:10]}")
print(f"Average linkage (K=2):  {clusters_average_2[:10]}")
print(f"Single linkage (K=2):   {clusters_single_2[:10]}")
print(f"Single linkage (K=4):   {clusters_single_4[:10]}")

# Cluster sizes
print(f"\nCluster sizes:")
print(f"Complete (K=2): {np.bincount(clusters_complete_2)[1:]}")
print(f"Average (K=2):  {np.bincount(clusters_average_2)[1:]}")
print(f"Single (K=2):   {np.bincount(clusters_single_2)[1:]}")
print(f"Single (K=4):   {np.bincount(clusters_single_4)[1:]}")

In [ ]:
# Visualize hierarchical clustering results
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# Complete linkage (K=2)
for i in [1, 2]:
    mask = clusters_complete_2 == i
    axes[0, 0].scatter(X[mask, 0], X[mask, 1], s=100, alpha=0.6, label=f'Cluster {i}')
axes[0, 0].set_title('Complete Linkage (K=2)', fontsize=12)
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Average linkage (K=2)
for i in [1, 2]:
    mask = clusters_average_2 == i
    axes[0, 1].scatter(X[mask, 0], X[mask, 1], s=100, alpha=0.6, label=f'Cluster {i}')
axes[0, 1].set_title('Average Linkage (K=2)', fontsize=12)
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Single linkage (K=2)
for i in [1, 2]:
    mask = clusters_single_2 == i
    axes[1, 0].scatter(X[mask, 0], X[mask, 1], s=100, alpha=0.6, label=f'Cluster {i}')
axes[1, 0].set_title('Single Linkage (K=2)', fontsize=12)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Single linkage (K=4)
colors = ['red', 'blue', 'green', 'orange']
for i in range(1, 5):
    mask = clusters_single_4 == i
    if mask.sum() > 0:
        axes[1, 1].scatter(X[mask, 0], X[mask, 1], c=colors[i-1], s=100, alpha=0.6, 
                          label=f'Cluster {i}')
axes[1, 1].set_title('Single Linkage (K=4)', fontsize=12)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Scaling Features Before Clustering

In [ ]:
# Scale features (mean=0, std=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Hierarchical clustering on scaled data
linkage_scaled = linkage(X_scaled, method='complete')

# Compare dendrograms
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Unscaled
dendrogram(linkage_complete, ax=axes[0], no_labels=True)
axes[0].set_title('Complete Linkage (Original Features)', fontsize=14)
axes[0].set_ylabel('Distance', fontsize=12)

# Scaled
dendrogram(linkage_scaled, ax=axes[1], no_labels=True)
axes[1].set_title('Complete Linkage (Scaled Features)', fontsize=14)
axes[1].set_ylabel('Distance', fontsize=12)

plt.tight_layout()
plt.show()

print("Scaling is important when variables have different scales!")
print("In this case, features already on similar scale, so little difference.")

## 5. Correlation-Based Distance

In [ ]:
# Generate data for correlation-based clustering
np.random.seed(1)
X_corr = np.random.randn(30, 3)

# Compute correlation-based distance
# Distance = 1 - correlation
corr_matrix = np.corrcoef(X_corr)
corr_dist = 1 - corr_matrix

# Convert to condensed distance matrix for linkage
from scipy.spatial.distance import squareform
corr_dist_condensed = squareform(corr_dist)

# Hierarchical clustering
linkage_corr = linkage(corr_dist_condensed, method='complete')

print(f"Correlation-based distance matrix shape: {corr_dist.shape}")
print(f"\nExample correlations (first 5x5):")
print(corr_matrix[:5, :5])

In [ ]:
# Plot dendrogram with correlation-based distance
plt.figure(figsize=(10, 6))
dendrogram(linkage_corr, no_labels=True)
plt.title('Complete Linkage with Correlation-Based Distance', fontsize=14)
plt.xlabel('')
plt.ylabel('1 - Correlation', fontsize=12)
plt.tight_layout()
plt.show()

print("Correlation-based distance:")
print("- Distance = 1 - correlation")
print("- Groups observations with similar patterns (regardless of magnitude)")
print("- Useful for gene expression, time series, etc.")

## 6. Application: NCI60 Cancer Gene Expression Data

64 cancer cell lines, ~6,000 genes.

In [ ]:
# Load NCI60 data (or create synthetic high-dimensional data)
# Real NCI60 has 64 cell lines, 6830 genes

# Create synthetic data similar to NCI60
np.random.seed(1)
n_samples = 64
n_genes = 6830

# Cancer types
cancer_types = ['CNS', 'RENAL', 'BREAST', 'NSCLC', 'UNKNOWN', 'OVARIAN', 
               'MELANOMA', 'PROSTATE', 'LEUKEMIA', 'K562B-repro', 'K562A-repro', 
               'MCF7A-repro', 'MCF7D-repro', 'COLON']

# Generate labels
nci_labs = np.random.choice(cancer_types, n_samples, 
                           p=[0.08, 0.14, 0.11, 0.14, 0.02, 0.11, 0.13, 0.03, 0.11, 0.02, 0.02, 0.02, 0.02, 0.11])

# Generate gene expression data (add signal for different cancer types)
nci_data = np.random.randn(n_samples, n_genes) * 0.5

# Add cancer-type-specific signal
for i, cancer in enumerate(np.unique(nci_labs)):
    mask = nci_labs == cancer
    gene_start = i * 500
    gene_end = min((i+1) * 500, n_genes)
    nci_data[mask, gene_start:gene_end] += np.random.randn(mask.sum(), gene_end - gene_start) * 2

print(f"NCI60 Cancer Gene Expression Data:")
print(f"Samples: {n_samples}")
print(f"Genes: {n_genes}")
print(f"\nCancer types (first 10): {nci_labs[:10]}")
print(f"\nCancer type distribution:")
for cancer in np.unique(nci_labs):
    print(f"  {cancer}: {(nci_labs == cancer).sum()}")

### PCA on Gene Expression Data

In [ ]:
# Perform PCA
pca = PCA()
pca.fit(nci_data)

# Transform data
pc_scores = pca.transform(nci_data)

print(f"PCA on Gene Expression Data:")
print(f"Number of principal components: {pca.n_components_}")
print(f"\nVariance explained by first 5 PCs:")
for i in range(5):
    print(f"  PC{i+1}: {pca.explained_variance_ratio_[i]*100:.2f}%")

cumulative_var = np.cumsum(pca.explained_variance_ratio_)
print(f"\nCumulative variance (first 5 PCs): {cumulative_var[4]*100:.2f}%")

In [ ]:
# Color function for cancer types
def get_colors(labels):
    unique_labels = np.unique(labels)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))
    color_map = dict(zip(unique_labels, colors))
    return [color_map[label] for label in labels]

colors = get_colors(nci_labs)

# Plot PC1 vs PC2 and PC1 vs PC3
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# PC1 vs PC2
axes[0].scatter(pc_scores[:, 0], pc_scores[:, 1], c=colors, s=80, alpha=0.7)
axes[0].set_xlabel('PC1', fontsize=12)
axes[0].set_ylabel('PC2', fontsize=12)
axes[0].set_title('NCI60: PC1 vs PC2', fontsize=14)
axes[0].grid(True, alpha=0.3)

# PC1 vs PC3
axes[1].scatter(pc_scores[:, 0], pc_scores[:, 2], c=colors, s=80, alpha=0.7)
axes[1].set_xlabel('PC1', fontsize=12)
axes[1].set_ylabel('PC3', fontsize=12)
axes[1].set_title('NCI60: PC1 vs PC3', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Similar cancer types cluster together in PC space!")

In [ ]:
# Scree plot and cumulative variance
pve = pca.explained_variance_ratio_ * 100
cumulative_pve = np.cumsum(pve)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scree plot
axes[0].plot(range(1, 21), pve[:20], marker='o', linewidth=2, color='blue')
axes[0].set_xlabel('Principal Component', fontsize=12)
axes[0].set_ylabel('PVE (%)', fontsize=12)
axes[0].set_title('Scree Plot (first 20 PCs)', fontsize=14)
axes[0].grid(True, alpha=0.3)

# Cumulative variance
axes[1].plot(range(1, 21), cumulative_pve[:20], marker='o', linewidth=2, color='brown')
axes[1].set_xlabel('Principal Component', fontsize=12)
axes[1].set_ylabel('Cumulative PVE (%)', fontsize=12)
axes[1].set_title('Cumulative Variance Explained (first 20 PCs)', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"First PC explains {pve[0]:.2f}% of variance")
print(f"First 5 PCs explain {cumulative_pve[4]:.2f}% of variance")

### Hierarchical Clustering on Gene Expression Data

In [ ]:
# Scale data
scaler = StandardScaler()
nci_scaled = scaler.fit_transform(nci_data)

# Compute linkages
linkage_nci_complete = linkage(nci_scaled, method='complete')
linkage_nci_average = linkage(nci_scaled, method='average')
linkage_nci_single = linkage(nci_scaled, method='single')

print("Hierarchical clustering on scaled gene expression data")

In [ ]:
# Plot dendrograms
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Complete linkage
dendrogram(linkage_nci_complete, ax=axes[0], labels=nci_labs, leaf_font_size=8)
axes[0].set_title('Complete Linkage', fontsize=14)
axes[0].set_xlabel('')
axes[0].set_ylabel('Distance', fontsize=12)

# Average linkage
dendrogram(linkage_nci_average, ax=axes[1], labels=nci_labs, leaf_font_size=8)
axes[1].set_title('Average Linkage', fontsize=14)
axes[1].set_xlabel('')
axes[1].set_ylabel('Distance', fontsize=12)

# Single linkage
dendrogram(linkage_nci_single, ax=axes[2], labels=nci_labs, leaf_font_size=8)
axes[2].set_title('Single Linkage', fontsize=14)
axes[2].set_xlabel('')
axes[2].set_ylabel('Distance', fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Cut dendrogram to get 4 clusters
hc_clusters = fcluster(linkage_nci_complete, 4, criterion='maxclust')

# Cross-tabulation: clusters vs cancer types
ct = pd.crosstab(hc_clusters, nci_labs, rownames=['Cluster'], colnames=['Cancer Type'])

print("Hierarchical Clustering (K=4) vs True Cancer Types:")
print(ct)

print(f"\nCluster sizes: {np.bincount(hc_clusters)[1:]}")

In [ ]:
# Dendrogram with cut line
plt.figure(figsize=(14, 6))
dendrogram(linkage_nci_complete, labels=nci_labs, leaf_font_size=10)
plt.axhline(y=139, color='red', linestyle='--', linewidth=2, label='Cut at K=4')
plt.xlabel('Cancer Cell Line', fontsize=12)
plt.ylabel('Distance', fontsize=12)
plt.title('Hierarchical Clustering Dendrogram (Complete Linkage)', fontsize=14)
plt.legend()
plt.tight_layout()
plt.show()

print("Cutting dendrogram at red line gives 4 clusters")

### Clustering on Principal Component Scores

In [ ]:
# Hierarchical clustering on first 5 principal components
linkage_pca = linkage(pc_scores[:, :5], method='complete')

plt.figure(figsize=(14, 6))
dendrogram(linkage_pca, labels=nci_labs, leaf_font_size=10)
plt.xlabel('Cancer Cell Line', fontsize=12)
plt.ylabel('Distance', fontsize=12)
plt.title('Hierarchical Clustering on First 5 PC Scores', fontsize=14)
plt.tight_layout()
plt.show()

# Cut to get 4 clusters
pca_clusters = fcluster(linkage_pca, 4, criterion='maxclust')

# Cross-tabulation
ct_pca = pd.crosstab(pca_clusters, nci_labs, rownames=['PC Cluster'], colnames=['Cancer Type'])

print("\nClustering on PC Scores (K=4) vs True Cancer Types:")
print(ct_pca)

print("\nBenefit: Much faster (5 features instead of 6830)!")
print("Similar clustering structure to using all genes.")

## Summary

This notebook covered:

### **K-Means Clustering**
- **Goal**: Partition n observations into K clusters
- **Algorithm**: Iteratively assign points to nearest centroid, update centroids
- **Key parameter**: K (number of clusters)
- **n_init**: Use multiple initializations to avoid local optima
- **Limitation**: Need to specify K in advance

### **Hierarchical Clustering**
- **Goal**: Build hierarchy of clusters (bottom-up)
- **Output**: Dendrogram showing clustering at all levels
- **Advantage**: Don't need to specify K in advance
- **Cut dendrogram** at different heights to get different K

### **Linkage Methods**
- **Complete**: Maximum distance between clusters (balanced)
- **Single**: Minimum distance (can produce chaining)
- **Average**: Average distance (compromise)
- **Ward**: Minimize within-cluster variance (often best)

### **Distance Metrics**
- **Euclidean**: Standard L2 distance
- **Correlation-based**: 1 - correlation (pattern similarity)
- **Manhattan, etc.**: Other options available

### **Best Practices**
- ✅ **Scale features** if on different scales
- ✅ Use **n_init >= 20** for K-means
- ✅ Try **multiple linkage methods**
- ✅ Use **PCA** for visualization and speed-up
- ✅ **Validate** with domain knowledge

### **Gene Expression Example**
- **High-dimensional data**: 64 samples, 6830 genes
- **PCA useful**: Reduces dimensionality, speeds up clustering
- **Hierarchical clustering**: Groups similar cancer types
- **Clustering on PC scores**: Similar results, much faster

### **When to Use Each Method**
- **K-Means**: Fast, good when K is known, spherical clusters
- **Hierarchical**: Unknown K, want hierarchy, any cluster shape
- **PCA + Clustering**: High-dimensional data, visualization

### **Key Takeaway**
Clustering is **exploratory** - use domain knowledge to interpret and validate results!